In [1]:
import os
import subprocess

In [ ]:
# VERSION SETTING
version_dict = {
    "original" : ["kd_tree", "angular"],
    "modified" : ["01_ring", "02_xyz", "03_group8"]
}

selected_type = "modified"
version = version_dict[selected_type][2]

print(f"Selected version: {version}")

Selected version: 03_group8


In [3]:
project_path = "/home/noh/pgc"

if version in version_dict["original"]:
    tmc3_exe = f"{project_path}/source/mpeg-pcc-tmc13/build/tmc3/tmc3"
    version_label = "original"
else:
    tmc3_exe = f"{project_path}/source/custom/build/{version}/{version}_tmc3"
    version_label = "modified"

source_path = "/home/noh/pgc/datasets/nuscenes/v1.0-mini/ply"
path = {
    "bin_ply": f"{source_path}/bin",
    "ascii_ply": f"{source_path}/ascii",
    "output_dir": f"{project_path}/experiments"
}

print(f"TMC3 Version: {version_label}, Selected Mode: {version}")
print(f"Executable Path: {tmc3_exe}")

TMC3 Version: modified, Selected Mode: 03_group8
Executable Path: /home/noh/pgc/source/custom/build/03_group8/03_group8_tmc3


### Predictive Geometry Coding (PGC) Encoder

In [4]:
config_label = "test2"
version = f"{version}_{config_label}"
print(version)

# TMC3 Settings
TMC3 = [
    "--mode=0", # Encoder
    "--srcUnit=metre",
    "--srcUnitLength=1",
    "--inputScale=100",
    "--mergeDuplicatedPoints=0"
]

# Geometry Settings (PGC)
PGC = [
    "--geomTreeType=1", # Prediction Tree
    "--trisoupNodeSizeLog2=0", # Disable trisoup
    "--adjacentChildContextualization=0", # To handle warning
    "--predGeomSort=1", # Sorting before the tree generation
    # f"--predGeomTreePtsMax={ring_points}", # Maximum number of points in the tree
    "--partitionMethod=0", # No partitioning
    # f"--sliceMaxPoints={ring_points}",
    # "--sliceMinPoints=0",
    "--interPredictionEnabled=0", # Intra Prediction
    "--biPredictionEnabled=0",
    "--globalMotionEnabled=0",
    
    ### Angular Mode Parameters
    "--angularEnabled=1",# PGC Angular Mode
    "--numLasers=32", # Number of channers (LiDAR)
    
    ## Laser Parameter Arrays, not used for ring-based mode 
    "--lasersTheta=-0.5235, -0.5010,  -0.4785, -0.4560, -0.4335, -0.4109, -0.3884, -0.3659, -0.3434, -0.3209, -0.2983, -0.2758, -0.2533, -0.2308, -0.2083, -0.1857, -0.1632, -0.1407, -0.1182, -0.0957,  -0.0731, -0.0506, -0.0281, -0.0056, 0.0168, 0.0394, 0.0619, 0.0844, 0.1069, 0.1294, 0.1520, 0.1745",
    "--lasersZ=0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0",
    "--lasersNumPhiPerTurn=1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090",
    
    "--enableGroundPartition=0", # Disable Ground Partition (default=0 but set it to be sure)
    "--predGeomAzimuthQuantization=0", # Azimuth Scaling Disabled 
    "--positionAzimuthSpeed=65538", # To set pgeom_resid_abs_log2_bits[1] as 5
    "--secondaryResidualDisabled=1", # Cartesian Compenstation after conversion

    ## Entropy Coding
    "--entropyContinuationEnabled=0", # Disable Entropy Context continuation between slices

    ## Output format (ascii | binary)
    "--outputBinaryPly=0",
]

ATTRIBUTE = [
    "--disableAttributeCoding=1" # Geometry Compression Only
]

config = TMC3 + PGC + ATTRIBUTE
config

03_group8_test2


['--mode=0',
 '--srcUnit=metre',
 '--srcUnitLength=1',
 '--inputScale=100',
 '--mergeDuplicatedPoints=0',
 '--geomTreeType=1',
 '--trisoupNodeSizeLog2=0',
 '--adjacentChildContextualization=0',
 '--predGeomSort=1',
 '--partitionMethod=0',
 '--interPredictionEnabled=0',
 '--biPredictionEnabled=0',
 '--globalMotionEnabled=0',
 '--angularEnabled=1',
 '--numLasers=32',
 '--lasersTheta=-0.5235, -0.5010,  -0.4785, -0.4560, -0.4335, -0.4109, -0.3884, -0.3659, -0.3434, -0.3209, -0.2983, -0.2758, -0.2533, -0.2308, -0.2083, -0.1857, -0.1632, -0.1407, -0.1182, -0.0957,  -0.0731, -0.0506, -0.0281, -0.0056, 0.0168, 0.0394, 0.0619, 0.0844, 0.1069, 0.1294, 0.1520, 0.1745',
 '--lasersZ=0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0',
 '--lasersNumPhiPerTurn=1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090, 1090',
 '--ena

In [5]:
def Encoder(tmc3, ply, bitstream, config, verbose=True):
    tmc3_exe = tmc3
    uncompressedPly = f"--uncompressedDataPath={ply}"
    compressedBitstream = f"--compressedStreamPath={bitstream}"
    reconstructedPly = f"--reconstructedDataPath={bitstream.replace('.bin', '_recon.ply')}"
    
    log_path = bitstream.replace('.bin', '.log')
    err_path = bitstream.replace('.bin', '.err')
    
    cmd = [tmc3_exe, uncompressedPly, compressedBitstream, reconstructedPly] + config
    try:
        terminal = subprocess.run(cmd, check=True, 
            stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout = terminal.stdout.decode('utf-8')
        stderr = terminal.stderr.decode('utf-8')
        
        if verbose:
            with open(log_path, 'w') as log_file:
                log_file.write(stdout)
            # Write stderr even on success (may contain warnings)
            if stderr:
                with open(err_path, 'w') as err_file:
                    err_file.write(stderr)
        
        return True
    
    except subprocess.CalledProcessError as exc:
        # Write both stdout and stderr on failure
        if verbose:
            # Write stdout if available
            if exc.stdout:
                with open(log_path, 'w') as log_file:
                    log_file.write(f"[FAILED - Return code: {exc.returncode}]\n")
                    log_file.write(exc.stdout.decode('utf-8'))
            
            # Write stderr with crash info
            with open(err_path, 'w') as err_file:
                err_file.write(f"[CRASH - Signal/Return code: {exc.returncode}]\n")
                err_file.write(f"Command: {' '.join(exc.cmd)}\n\n")
                if exc.stderr:
                    err_file.write(exc.stderr.decode('utf-8'))
                else:
                    err_file.write("No stderr output (process terminated abnormally)\n")
        
        return False

In [6]:
if os.path.exists(path["bin_ply"]):
    total_files = 0
    success_files = 0
    error_files = 0
    print("="*50)
    print("ENCODING STARTS")
    print("="*50)
    tmc3_encoder = tmc3_exe
    for scene in os.listdir(path["bin_ply"]):
        print(f"[{scene} is found.]")
        scene_path = os.path.join(path["bin_ply"], scene)

        if not os.path.isdir(scene_path):
            continue

        if version:
            scene_bitstream_path = os.path.join(path["output_dir"], version, scene)
            # tmc3_encoder = sandbox_exe
        else:
            scene_bitstream_path = os.path.join(path["output_dir"], "original", scene)
            # tmc3_encoder = original_exe
        print(f">> Checking Encoder: {tmc3_encoder}")
        print(f">> Exists: {os.path.exists(tmc3_encoder)}")
        os.makedirs(scene_bitstream_path, exist_ok=True)
        
        frame_count = 0
        frame_success_files = 0
        for frame in os.listdir(scene_path):
            if frame.endswith(".ply"):
                total_files += 1
                frame_count += 1

                uncompressedPly = os.path.join(scene_path, frame)
                compressedBitstream = os.path.join(scene_bitstream_path, frame.replace(".ply", ".bin"))
                
                result = Encoder(tmc3_encoder, uncompressedPly, compressedBitstream, config)
                if result:
                    success_files += 1
                    frame_success_files += 1
                    print(f"    [{frame} is encoded successfully]")
                else:
                    error_files += 1
                    print(f"    [{frame} is failed to encode]")
        print(f"    >> {scene}: {frame_success_files}/{frame_count} frames are encoded.\n")
    print("="*50)
    print(f"Total {total_files} files are processed.")
    print(f"Success: {success_files}")
    print(f"Error: {error_files}")
    print("="*50)
        

ENCODING STARTS
[scene-0916 is found.]
>> Checking Encoder: /home/noh/pgc/source/custom/build/03_group8/03_group8_tmc3
>> Exists: True
    [scene-0916_01.ply is encoded successfully]
    [scene-0916_18.ply is encoded successfully]
    [scene-0916_35.ply is encoded successfully]
    [scene-0916_20.ply is encoded successfully]
    [scene-0916_19.ply is encoded successfully]
    [scene-0916_26.ply is encoded successfully]
    [scene-0916_30.ply is encoded successfully]
    [scene-0916_03.ply is encoded successfully]
    [scene-0916_33.ply is encoded successfully]
    [scene-0916_08.ply is encoded successfully]
    [scene-0916_31.ply is encoded successfully]
    [scene-0916_15.ply is encoded successfully]
    [scene-0916_24.ply is encoded successfully]
    [scene-0916_36.ply is encoded successfully]
    [scene-0916_28.ply is encoded successfully]
    [scene-0916_17.ply is encoded successfully]
    [scene-0916_25.ply is encoded successfully]
    [scene-0916_12.ply is encoded successfully]
 